# Portfolio construction model trading strategy example

This is an example notebook how to construct a momentum based portfolio construction strategy
using [Trading Strategy framework](https://tradingstrategy.ai/docs/) and backtest it for DeFi tokens.

This backtest uses alpha model approach where each trading pair has a signal
and basede on the signal strenghts we construct new portfolio weightings 
for the upcoming week.

Some highlights of this notebook:

* **Not a realistic trading strategy, but more of an code example** - this may generate profits or loss but this is outside the scode of this example
* Make sure you have studied some simpler backtesting examples first
* The backtest has all its code within a single Jupyter notebook
    - The backtest code and charts are self-contained in a single file
    - The example code is easy to read
* Runs a backtest for a momentum strategy
    - Uses handpicked "DeFi bluechip tokens" - see *Trading universe definition* for the list
    - Long only    
    - Pick top 3 tokens for each strategy cycle
    - Trade across multiple blockchains
    - Trade across multiple DEXes
    - Based on momentum (previous week price change %)
    - Weekly rebalances - weekly strategy cycle
    - Supports stop loss - although not used in this example
    - Ignore price impact, and thus may cause unrealistic results
    - Ignore available liquidity to trade, and thus may cause unrealistic results
    - Ignore any complications of trading assets cross chain - we assume our reserve currency USDC is good on any chain and DEX and trading pair,
      which is not a realistic assumption
* Demostrates statistics and performance analyses
    - Equity curve with comparison to buy and hold ETH
    - Alpha model 
    - Aggregated monthly performance
* Order routing is ignored; the strategy cannot be moved to live trading as is
    - We use the oldest available DEX price data we have: Uniswap v2, others
    - For live trade execution, one would want to choose execution on a newer DEX with less fees
* You need a [Trading Strategy API key](https://tradingstrategy.ai/trading-view/backtesting) to run the notebook

There is also a unit test version availble in the repository in `tests/strategy_tests`.


## Strategy parameter set up

Set up the parameters used in in this strategy backtest study.

- Backtested blockchain, exchange and trading pair
- Backtesting period
- Strategy parameters for EMA crossovers

In [1]:
import datetime
import pandas as pd

from tradingstrategy.chain import ChainId
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.strategy.strategy_module import StrategyType, TradeRouting, ReserveCurrency

# Tell what trade execution engine version this strategy needs to use
trading_strategy_engine_version = "0.1"

# What kind of strategy we are running.
# This tells we are going to use
trading_strategy_type = StrategyType.managed_positions

# How our trades are routed.
# PancakeSwap basic routing supports two way trades with BUSD
# and three way trades with BUSD-BNB hop.
trade_routing = TradeRouting.ignore

# Set cycle to 7 days and look back the momentum of the previous candle
trading_strategy_cycle = CycleDuration.cycle_7d
momentum_lookback_period = datetime.timedelta(days=7)

# Hold top 3 coins for every cycle
max_assets_in_portfolio = 4

# Leave 20% cash buffer
value_allocated_to_positions = 0.80

# Set 2% midprice stop loss
stop_loss = 0.66

take_profit = 1.05

# Use hourly candles to trigger the stop loss
stop_loss_data_granularity = TimeBucket.h1

# Strategy keeps its cash in USDC
reserve_currency = ReserveCurrency.usdc

# The duration of the backtesting period
start_at = datetime.datetime(2020, 11, 1)
end_at = datetime.datetime(2023, 1, 31)

# Start with 10,000 USD
initial_deposit = 10_000


## Strategy logic and trade decisions

`decide_trades` function decide what trades to take.
In this example, we calculate two exponential moving averages
(EMAs) and make decisions based on those.


In [2]:
from tradeexecutor.strategy.trading_strategy_universe import translate_trading_pair
from typing import List, Dict, Counter

from tradingstrategy.universe import Universe
from tradeexecutor.strategy.weighting import weight_by_1_slash_n
from tradeexecutor.strategy.alpha_model import AlphaModel
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pricing_model import PricingModel
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradeexecutor.state.state import State


def decide_trades(
        timestamp: pd.Timestamp,
        universe: Universe,
        state: State,
        pricing_model: PricingModel,
        cycle_debug_data: Dict) -> List[TradeExecution]:

    # Create a position manager helper class that allows us easily to create
    # opening/closing trades for different positions
    position_manager = PositionManager(timestamp, universe, state, pricing_model)

    alpha_model = AlphaModel(timestamp)

    # Watch out for the inclusive range and include and avoid peeking in the future
    adjusted_timestamp = timestamp - pd.Timedelta(seconds=1)
    start = adjusted_timestamp - momentum_lookback_period - datetime.timedelta(seconds=1)
    end = adjusted_timestamp 

    candle_universe = universe.candles
    pair_universe = universe.pairs

    # Get candle data for all candles, inclusive time range
    candle_data = candle_universe.iterate_samples_by_pair_range(start, end)

    # Iterate over all candles for all pairs in this timestamp (ts)
    for pair_id, pair_df in candle_data:

        # We should have candles for range start - end,
        # where end is the current strategy cycle timestamp 
        # and start is one week before end.
        # Because of sparse data we may have 0, 1 or 2 candles
        first_candle = pair_df.iloc[0]
        last_candle = pair_df.iloc[-1]

        # How many candles we are going to evaluate
        candle_count = len(pair_df)

        assert last_candle["timestamp"] < timestamp, "Something wrong with the data - we should not be able to peek the candle of the current timestamp, but always use the previous candle"

        open = last_candle["open"]
        close = last_candle["close"]

        # DEXPair instance contains more data than internal TradingPairIdentifier
        # we use to store this pair across the strategy
        dex_pair = pair_universe.get_pair_by_id(pair_id)
        pair = translate_trading_pair(dex_pair)

        # We define momentum as how many % the trading pair price gained during
        # the momentum window
        momentum = (close - open) / open
        
        # This pair has not positive momentum,
        # we only buy when stuff goes up
        if momentum <= 0.025:
            continue

        alpha_model.set_signal(
            pair,
            momentum,
            stop_loss=stop_loss,
            take_profit=take_profit,
        )

    # Select max_assets_in_portfolio assets in which we are going to invest
    # Calculate a weight for ecah asset in the portfolio using 1/N method based on the raw signal
    alpha_model.select_top_signals(max_assets_in_portfolio)
    alpha_model.assign_weights(method=weight_by_1_slash_n)
    alpha_model.normalise_weights()

    # Load in old weight for each trading pair signal,
    # so we can calculate the adjustment trade size
    alpha_model.update_old_weights(state.portfolio)

    # Calculate how much dollar value we want each individual position to be on this strategy cycle,
    # based on our total available equity
    portfolio = position_manager.get_current_portfolio()
    portfolio_target_value = portfolio.get_total_equity() * value_allocated_to_positions
    alpha_model.calculate_target_positions(portfolio_target_value)

    # Shift portfolio from current positions to target positions
    # determined by the alpha signals (momentum)
    trades = alpha_model.generate_adjustment_trades_and_update_stop_losses(position_manager)

    # Record alpha model state so we can later visualise our alpha model thinking better
    state.visualisation.add_calculations(timestamp, alpha_model.to_dict())

    return trades


## Set up the market data client

The [Trading Strategy market data client](https://tradingstrategy.ai/docs/programming/api/client/index.html)
is the Python library responsible for managing the data feeds needed to run the backtest.None

We set up the market data client with an API key.

[If you do not have an API key yet, you can register one](https://tradingstrategy.ai/trading-view/backtesting).

In [3]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


## Setup trading universe

We setup the trading universe for the backtesting.

- Read in a handwritten allowed trading pair universe list

- Download candle data

- Print out trading pair addresses and volumes as the sanity check the pair defintions look correct


In [4]:
from tradingstrategy.client import Client

from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse
from tradeexecutor.strategy.trading_strategy_universe import load_partial_data
from tradeexecutor.strategy.execution_context import ExecutionContext
from tradeexecutor.strategy.execution_context import ExecutionMode
from tradeexecutor.strategy.universe_model import UniverseOptions


# List of trading pairs that we consider "DeFi blueschips" for this strategy
# For token ordering, wrappign see https://tradingstrategy.ai/docs/programming/market-data/trading-pairs.html
pairs = (
    (ChainId.ethereum, "uniswap-v2", "WETH", "USDC"),  # ETH
    (ChainId.ethereum, "sushi", "AAVE", "WETH"),  # AAVE
    (ChainId.ethereum, "uniswap-v2", "UNI", "WETH"),  # UNI
    (ChainId.ethereum, "uniswap-v2", "CRV", "WETH"),  # Curve
    (ChainId.ethereum, "sushi", "SUSHI", "WETH"),  # Sushi
    (ChainId.bsc, "pancakeswap-v2", "WBNB", "BUSD"),  # BNB
    (ChainId.bsc, "pancakeswap-v2", "Cake", "BUSD"),  # Cake
    (ChainId.polygon, "quickswap", "WMATIC", "USDC"),  # Matic
    (ChainId.avalanche, "trader-joe", "WAVAX", "USDC"),  # Avax
    (ChainId.avalanche, "trader-joe", "JOE", "WAVAX"),  # TraderJoe
)


def create_trading_universe(
        ts: datetime.datetime,
        client: Client,
        execution_context: ExecutionContext,
        universe_options: UniverseOptions,
) -> TradingStrategyUniverse:

    assert not execution_context.mode.is_live_trading(), \
        f"Only strategy backtesting supported, got {execution_context.mode}"

    # Load data for our trading pair whitelist
    dataset = load_partial_data(
        client=client,
        time_bucket=trading_strategy_cycle.to_timebucket(),
        pairs=pairs,
        execution_context=execution_context,
        universe_options=universe_options,
        liquidity=False,
        stop_loss_time_bucket=stop_loss_data_granularity,
        start_at=start_at,
        end_at=end_at,
    )

    # Filter down the dataset to the pairs we specified
    universe = TradingStrategyUniverse.create_multichain_universe_by_pair_descriptions(
        dataset,
        pairs,
        reserve_token_symbol="USDC"  # Pick any USDC - does not matter as we do not route
    )

    return universe

universe = create_trading_universe(
    datetime.datetime.utcnow(),
    client,
    ExecutionContext(mode=ExecutionMode.backtesting),
    UniverseOptions(),
)

for pair in universe.universe.pairs.iterate_pairs():
    all_time_volume = pair.buy_volume_all_time
    print(f"Trading pair {pair.base_token_symbol} ({pair.base_token_address}) - {pair.quote_token_symbol} ({pair.quote_token_address}) - all time volume {all_time_volume:,.0f} USD")

Trading pair WETH (0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2) - USDC (0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48) - all time volume 21,349,838,848 USD
Trading pair CRV (0xd533a949740bb3306d119cc777fa900ba034cd52) - WETH (0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2) - all time volume 580,417,920 USD
Trading pair SUSHI (0x6b3595068778dd592e39a122f4f5a5cf09c90fe2) - WETH (0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2) - all time volume 7,749,442,048 USD
Trading pair UNI (0x1f9840a85d5af5bf1d1762f925bdaddc4201f984) - WETH (0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2) - all time volume 3,978,604,032 USD
Trading pair AAVE (0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9) - WETH (0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2) - all time volume 1,641,289,856 USD
Trading pair WMATIC (0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270) - USDC (0x2791bca1f2de4661ed88a30c99a7a9449aa84174) - all time volume 2,986,208,000 USD
Trading pair Cake (0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82) - BUSD (0xe9e7cea3dedca5984780b

## Run backtest

Run backtest using giving trading universe and strategy function.

- Running the backtest outputs `state` object that contains all the information
on the backtesting position and trades.
- The trade execution engine will download the necessary datasets to run the backtest.
  The datasets may be large, several gigabytes.


In [5]:
import logging

from tradeexecutor.backtest.backtest_runner import run_backtest_inline

state, universe, debug_dump = run_backtest_inline(
    name="DeFi bluechip portfolio construction model example",
    start_at=start_at,
    end_at=end_at,
    client=client,
    cycle_duration=trading_strategy_cycle,
    decide_trades=decide_trades,
    create_trading_universe=create_trading_universe,
    initial_deposit=initial_deposit,
    reserve_currency=reserve_currency,
    trade_routing=trade_routing,
    log_level=logging.WARNING,
    universe=universe,
)

trade_count = len(list(state.portfolio.get_all_trades()))
print(f"Backtesting completed, backtested strategy made {trade_count} trades")

  0%|          | 0/70934400 [00:00<?, ?it/s]

Backtesting completed, backtested strategy made 592 trades


## Examine backtest results

Examine `state` that contains all actions the trade executor took.

We plot out a chart that shows
- The price action
- When the strategy made buys or sells

In [6]:
print(f"Positions taken: {len(list(state.portfolio.get_all_positions()))}")
print(f"Trades made: {len(list(state.portfolio.get_all_trades()))}")

Positions taken: 298
Trades made: 592


## Benchmarking the strategy performance

Here we benchmark the strategy performance against some baseline scenarios.

- Buy and hold ETH
- Buy and hold US Dollar (do nothing)


In [7]:
from tradeexecutor.visual.benchmark import visualise_benchmark

eth_usd = universe.get_pair_by_human_description((ChainId.ethereum, "uniswap-v2", "WETH", "USDC"))
eth_candles = universe.universe.candles.get_candles_by_pair(eth_usd.internal_id)
eth_price = eth_candles["close"]

fig = visualise_benchmark(
    state.name,
    portfolio_statistics=state.stats.portfolio,
    all_cash=state.portfolio.get_initial_deposit(),
    buy_and_hold_asset_name="ETH",
    buy_and_hold_price_series=eth_price,
    start_at=start_at,
    end_at=end_at
)

fig.show()

## Analysing  the strategy success

Here we calculate statistics on how well the strategy performed.

- Won/lost trades
- Timeline of taken positions with color coding of trade performance


In [8]:
from tradeexecutor.analysis.trade_analyser import build_trade_analysis

analysis = build_trade_analysis(state.portfolio)

### Strategy summary

Overview of strategy performance



In [9]:
from IPython.core.display_functions import display

summary = analysis.calculate_summary_statistics()

with pd.option_context("display.max_row", None):
    display(summary.to_dataframe())

,0
Trading period length,812 days
Return %,-33.30%
Annualised return %,-14.97%
Cash at start,"$10,000.00"
Value at end,"$6,669.73"
Trade win percent,60.54%
Total trades done,294
Won trades,178
Lost trades,116
Stop losses triggered,102


### Alpha model timeline analysis

We display a table that shows how alpha model performermed over time.

- Events are in the timeline order

- Display portfolio weightings at each strategy cycle

- Show performance of individidual positions taken during this cycle

- Stop loss events are shown as their own rows

The visualisation will have the a column for each asset being traded. The cell of a column contains the following elements

- Asset **price change** (up/down) since the last strategy cycle

- **Price** of the asset at the start of the strategy cycle

- **Weight** % as the part of the portfolio that was chosen for the next cycle

- **Open** value of the position as USD

- **Adjust** value of the position as USD -  to get to the target weight - how much asset needs to be bought or sold

- **Close** the remaining value of the position as USD 

- **Profit**: The life time profit of the position so far

Position opening and closing rules

- A position may contain multiple trades

- Unlike on some exchanges, we adjust existing positions by increasing or decreasing them - a trade can count against existing position

- Posions are opened if there are no existing holdings of an asset

- The position is adjusted over the time

- The position is closed when the remaining assets of the position are sold

In [10]:
from tradeexecutor.analysis.alpha_model_analyser import render_alpha_model_plotly_table, create_alpha_model_timeline_all_assets

timeline = create_alpha_model_timeline_all_assets(state, universe, new_line="<br>")
figure, table = render_alpha_model_plotly_table(timeline)

figure.update_layout(
    autosize=True,
    height=800,)

display(figure)

## Finishing notes

Print out a line to signal the notebook finished the execution successfully.

In [11]:
print("All ok")

All ok
